In [53]:
import pandas as pd
from numpy import round

In [14]:
data = pd.read_csv('레드 큐브.csv')

first = data.iloc[:, [0, 1]].copy()
first = first.loc[first.isnull().sum(axis = 1) == 0, :].set_index('Option 1').reset_index(drop = False)
second = data.iloc[:, [2, 3]].set_index('Option 2').reset_index(drop = False)
third = data.iloc[:, [4, 5]].set_index('Option 3').reset_index(drop = False)

In [55]:
def clean_chart(chart, i):
    option_and_value = chart.loc[:, f'Option {i}']
    chart = pd.concat([\
        pd.DataFrame([_ for _ in option_and_value.str.replace(' : ', ':').str.split(':')], columns = [f'Option {i}', f'Value {i}']), \
        chart.iloc[:, -1]
        ],
        axis = 1)
    
    str_to_float_indices = chart.iloc[:, 1].str.contains('%')

    chart.loc[str_to_float_indices, f'Value {i}'] = chart.loc[str_to_float_indices, f'Value {i}'].str.replace('%', '')
    chart.iloc[:, 1] = chart.iloc[:, 1].astype('float')
    chart.loc[str_to_float_indices, f'Value {i}'] *= 0.01
    chart.iloc[:, 1] = np.round(chart.iloc[:, 1], 2)
    
    chart.loc[str_to_float_indices, f'Option {i}'] = chart.loc[str_to_float_indices, f'Option {i}'] + '%'

    return chart.values.tolist()


def results_to_dict(result1, result2, result3):
    option1, val1, prob1 = result1
    result_dict = {}
    result_prob = prob1

    result_dict[option1] = val1

    for option, value, prob in (result2, result3):
        result_prob *= prob

        if option in result_dict.keys() and option == '몬스터 방어율 무시%':
            result_dict[option] = round(1 - (1 - result_dict[option]) * (1 - value))
        
        elif option in result_dict.keys() and option != '몬스터 방어율 무시%':
            result_dict[option] += value
            result_dict[option] = round(result_dict[option], 2)

        else:
            result_dict[option] = value
    
    result_dict = dict(sorted(result_dict.items()))
    return result_dict, result_prob

In [16]:
first = clean_chart(first, 1)
second = clean_chart(second, 2)
third = clean_chart(third, 3)

In [56]:
all_cases = [(result1, result2, result3) for result1 in first for result2 in second for result3 in third]
all_results = [results_to_dict(*case) for case in all_cases]
results, probs = zip(*all_results)

In [64]:
target_option = '공격력%'
target_value = 0.30
final_prob = 0

for idx, result in enumerate(results):
    if target_option in result.keys() and result[target_option] >= target_value:
        print(result)
        final_prob += probs[idx]

print(f'{round(final_prob * 100, 10)}%')

{'공격력%': 0.3}
{'공격력%': 0.33}
{'공격력%': 0.33}
{'공격력%': 0.36}
0.0229603598%
